## Prepare environment

In [ ]:
import os

ORIGINAL_DIRECTORY = None

def prepare_env():
    global ORIGINAL_DIRECTORY
    if ORIGINAL_DIRECTORY is None:
        ORIGINAL_DIRECTORY = os.getcwd()
    %cd {ORIGINAL_DIRECTORY}
    running_locally = os.path.isfile('./python-code')
    if not running_locally:
        if not os.path.isdir('./voxel-nn'):
            !git clone -b dev https://github.com/Sotakebk/voxel-nn.git
        %cd ./voxel-nn/Python
        !git pull
    
    %pip install -r ./requirements.txt
    %load_ext autoreload
    %autoreload 2
prepare_env()

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

import tensorflow as tf
from voxelnn import tensorflow_tools as tft
import os

if tft.maybe_use_tpu() is False:
    gpu = tf.config.list_physical_devices('GPU')
    print(gpu)

## Prepare dataset for further operations

In [ ]:
from voxelnn import dataset_manipulation as dm

shared_model_path = 'simple-terrain-2d-16x16'

files_to_load = [
    'simple-terrain-2d-16x16-dataset.json'
]

files_to_load = [f'../Dataset/{filename}' for filename in files_to_load]

tag_names, block_names, entry_names, tags_multilabel, blocks = dm.load_dataset(files_to_load)
print(f'Blocks: {block_names} ({len(block_names)})')
print(f'Tags: {tag_names} ({len(tag_names)})')
print(f'Data shape: {blocks.shape}')

In [ ]:
import seaborn as sns

tag_cooccurence_matrix = dm.cooccurence_matrix(tags_multilabel)
_ = sns.heatmap(data=tag_cooccurence_matrix, vmin=0, annot=True, xticklabels=tag_names, yticklabels=tag_names)

In [ ]:
import numpy as np
import seaborn as sns

unique, counts = np.unique(blocks, return_counts=True)

ax = sns.barplot(x=[block_names[u] for u in unique], y=counts)
_ = ax.set_title("Occurence of block types in the dataset.")

## Shared variables

In [ ]:
real_dims = len(blocks.shape)-1 # 2D or 3D
block_types = len(block_names) # number of separate classes
latent_dims = int(block_types**0.5) # how many dimensions to encode the block types in
print(f'real dimensions in data: {real_dims}')
print(f'block types in data: {block_types}')
print(f'latent dimensions to encode blocks in: {latent_dims}')

## Encoders

In [ ]:
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Input, Dense, Flatten, Conv3D, Conv2D, MaxPool3D, Dropout, Conv3DTranspose, Reshape, Lambda, UpSampling3D, ZeroPadding3D, CategoryEncoding, Softmax
from keras.models import Model, Sequential
from keras.utils import plot_model
from voxelnn.coding.custom_layers import OneHot, Sampling
#3d
def create_vae_encoder3d(dimensions, latent_dimensions):
    input = Input(shape=(None, None, None), dtype='uint8', name='encoder_input')
    oh = OneHot(dimensions)(input)
    c1  = Conv3D(10, kernel_size=1, activation='elu', padding='same', kernel_initializer='glorot_normal')(oh)
    c2  = Conv3D(20, kernel_size=1, activation='elu', padding='same', kernel_initializer='glorot_normal')(c1)
    c3  = Conv3D(20, kernel_size=1, activation='elu', padding='same', kernel_initializer='glorot_normal')(c2)
    mean = Conv3D(latent_dimensions, kernel_size=1, activation='linear', name='mean')(c2)
    var = Conv3D(latent_dimensions, kernel_size=1, activation='linear', kernel_initializer='zeros', name='var')(c3)
    z = Sampling(name='encoder_output')([mean, var])
    encoder = keras.Model(input, [mean, var, z], name="encoder")
    return encoder

def create_vae_decoder3d(dimensions, latent_dimensions):
    input = Input(shape=(None, None, None, latent_dimensions), name='decoder_input')
    c1 = Conv3D(10, kernel_size=1, activation='elu')(input)
    c2 = Conv3D(20, kernel_size=1, activation='elu')(c1)
    c3 = Conv3D(20, kernel_size=1, activation='elu')(c2)
    c4 = Conv3D(dimensions, kernel_size=1, activation='linear', name='decoder_output')(c3)
    sm = Softmax(axis=4)(c4)
    decoder = keras.Model(input, sm, name="decoder")
    return decoder

def create_vae_encoder2d(dimensions, latent_dimensions):
    input = Input(shape=(None, None), dtype='uint8', name='encoder_input')
    oh = OneHot(dimensions)(input)
    c1  = Conv2D(10, kernel_size=1, activation='elu', padding='same', kernel_initializer='glorot_normal')(oh)
    c2  = Conv2D(20, kernel_size=1, activation='elu', padding='same', kernel_initializer='glorot_normal')(c1)
    c3  = Conv2D(20, kernel_size=1, activation='elu', padding='same', kernel_initializer='glorot_normal')(c2)
    mean = Conv2D(latent_dimensions, kernel_size=1, activation='linear', name='mean')(c2)
    var = Conv2D(latent_dimensions, kernel_size=1, activation='linear', kernel_initializer='zeros', name='var')(c3)
    z = Sampling(name='encoder_output')([mean, var])
    encoder = keras.Model(input, [mean, var, z], name="encoder")
    return encoder

def create_vae_decoder2d(dimensions, latent_dimensions):
    input = Input(shape=(None, None, latent_dimensions), name='decoder_input')
    c1 = Conv2D(10, kernel_size=1, activation='elu')(input)
    c2 = Conv2D(20, kernel_size=1, activation='elu')(c1)
    c3 = Conv2D(20, kernel_size=1, activation='elu')(c2)
    c4 = Conv2D(dimensions, kernel_size=1, activation='linear', name='decoder_output')(c3)
    sm = Softmax(axis=3)(c4)
    decoder = keras.Model(input, sm, name="decoder")
    return decoder

In [ ]:
from voxelnn.coding import SAE
from voxelnn import filter_kernels as fk
from voxelnn import jupyter_tools as jt

e = create_vae_encoder2d(block_types, latent_dims)
d = create_vae_decoder2d(block_types, latent_dims)
e.compile()
d.compile()
vae = SAE.SAETrainer(e, d,
                     filter_array=fk.create_kernel(3, real_dims, latent_dims),
                     kld_loss_weight=0.1,
                     str_loss_weight=0.1)
vae.compile()

In [ ]:
split = int(0.9 * blocks.shape[0])
print(f'training on {split} entries')

history = vae.fit(blocks[:split,...], epochs=100, validation_data=(blocks[split:,...], None))

jt.plot_history(history)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def display_some_encoded_samples(with_sampling: bool):
  rows = 2
  columns = 5
  size_per_unit = 4
  z_mean, _, z = e.predict(blocks[(rows*columns):,...])
  data = z_mean if with_sampling else z
  new_array = np.zeros((*data.shape[:3], 3))
  new_array[:, :, :, :2] = data
  min_val = np.min(new_array)
  max_val = np.max(new_array)
  print((min_val, max_val))
  normalized_array = (new_array - min_val) / (max_val - min_val)
  print(f'min, max: {(min_val, max_val)}')
  f, axarr = plt.subplots(rows, columns, sharex=True, sharey=True, figsize=(columns * size_per_unit, rows * size_per_unit)) 
  for i in range(rows*columns):
    axarr[i//columns, i%columns].imshow(normalized_array[i])
    axarr[i//columns, i%columns].axis('off')

In [ ]:
if real_dims == 2:
  display_some_encoded_samples(with_sampling=True)

In [ ]:

if real_dims == 2:
  display_some_encoded_samples(with_sampling=False)

In [ ]:
z_mean, _, z = e.predict(blocks)

In [ ]:
normalization = keras.layers.Normalization(axis=-1)
normalization.adapt(z_mean)
print(f'mean: {normalization.mean}')
print(f'variance: {normalization.variance}')

norm_z = normalization(z, training=False)

denorm_z = normalization.mean + norm_z * normalization.variance ** 0.5

data = norm_z
new_array = np.zeros((*data.shape[:3], 3))
new_array[:, :, :, :2] = data
min_val = np.min(new_array)
max_val = np.max(new_array)
print((min_val, max_val))
normalized_array = (new_array - min_val) / (max_val - min_val)
new_array[:,:,:,[2]] = 0
f, axarr = plt.subplots(10, 5, sharex=True, sharey=True, figsize=(20, 40)) 
for i in range(50):
  axarr[i//5, i%5].imshow(normalized_array[i])
  axarr[i//5, i%5].axis('off')

In [ ]:
from voxelnn import jupyter_tools as jt
jt.save_model(shared_model_path, "encoder.keras", e)
jt.save_model(shared_model_path, "decoder.keras", d)

In [ ]:
from voxelnn import jupyter_tools as jt

e = jt.load_model(shared_model_path, "encoder.keras")
d = jt.load_model(shared_model_path, "decoder.keras")

## Diffusion

In [ ]:
from voxelnn.diffusion import unet_builder

layer_base_units = 32

unet = unet_builder.build_model(input_shape=blocks.shape[1:],
                          latent_dims=latent_dims,
                          layer_units=[a * layer_base_units for a in [1, 2, 4, 6]],
                          layer_has_attention=[False, False, True, True],
                          layer_res_blocks=[2, 2, 2, 2],
                          norm_groups=8,
                          first_conv_channels=32,
                          time_embedding_dims=32,
                          max_time_emb_frequency=200.0)
unet.compile()

In [ ]:
from voxelnn.diffusion.diffusion_model import DiffusionModel
import keras

diffusion_model = DiffusionModel(unet)
diffusion_model.compile(
        optimizer=keras.optimizers.AdamW(
        learning_rate=1e-3, weight_decay=1e-4
    ),
    loss=keras.losses.mean_absolute_error,
)

In [ ]:
z_mean, z_var, z = e.predict(np.repeat(np.concatenate([blocks, np.flip(blocks, axis=1)], axis=0), 10, axis=0))

In [ ]:
diffusion_model.learn_data_distribution(z)

In [ ]:
history = diffusion_model.fit(z, epochs=100)

jt.plot_history(history)

In [ ]:
import voxelnn.jupyter_tools as jt

jt.save_model(shared_model_path, "diffusion.keras", diffusion_model)

In [ ]:
import voxelnn.jupyter_tools as jt

diffusion_model = jt.load_model(shared_model_path, "diffusion.keras")

In [ ]:
result, data_history, noises_history, noisy_data_history = diffusion_model.generate_one_sample_with_history(diffusion_steps=50)

In [ ]:
import json
import keras
import numpy as np
import voxelnn.dataset_manipulation as dm

def decode_many_to_json(latent_data: np.ndarray, decoder: keras.Model, name_pattern: str = None):
    elements = latent_data.shape[0]
    name_pattern = name_pattern or f"Unnamed element ({{}}/{elements})"
    
    entry_list = []
    decoded = decoder.predict(latent_data)
    real = np.argmax(decoded, axis=-1)

    for i in range(elements):
        entry = dm.construct_entry_dto(
            str.format(name_pattern, i+1),
            tag_names,
            np.zeros(len(tag_names)),
            block_names,
            real[i,:])
        entry_list.append(entry)
    return json.dumps(entry_list)

In [ ]:
print("result:")
voxels_result = decode_many_to_json(result, d)
jt.result_json_widget(voxels_result)

In [ ]:
print("noisy data history:")
voxels_noisy_data_history = decode_many_to_json(noisy_data_history, d)
jt.result_json_widget(voxels_noisy_data_history)

In [ ]:
print("data history:")
voxels_data_history = decode_many_to_json(data_history, d)
jt.result_json_widget(voxels_data_history)

In [ ]:
import matplotlib.pyplot as plt

# render data_history

new_array = np.zeros((*data_history.shape[:3], 3))
new_array[:, :, :, :2] = data_history
min_val = np.min(new_array)
max_val = np.max(new_array)
print((min_val, max_val))
normalized_array = (new_array - min_val) / (max_val - min_val)
f, axarr = plt.subplots(10, 5, sharex=True, sharey=True, figsize=(20, 40)) 
for i in range(50):
  axarr[i//5, i%5].imshow(normalized_array[i])
  axarr[i//5, i%5].axis('off')

In [ ]:
import matplotlib.pyplot as plt

# render noisy_data_history

new_array = np.zeros((*noisy_data_history.shape[:3], 3))
new_array[:, :, :, :2] = noisy_data_history
min_val = np.min(new_array)
max_val = np.max(new_array)
print((min_val, max_val))
normalized_array = (new_array - min_val) / (max_val - min_val)
f, axarr = plt.subplots(10, 5, sharex=True, sharey=True, figsize=(20, 40)) 
for i in range(50):
  axarr[i//5, i%5].imshow(normalized_array[i])
  axarr[i//5, i%5].axis('off')